In [1]:
import os
import numpy as np
import quaternion
import time
from PIL import Image

import matplotlib.pyplot as plt
%matplotlib inline

import habitat

import torch.nn as nn
import torch.nn.functional as F
import torch
from torchvision.transforms import ToTensor
from torchvision.transforms.functional import rotate as torchrot
from models import device, WarpSeg, tensor
from utils import d3_40_colors_rgb

In [2]:
config = habitat.get_config("/home/azav/habitat-api/configs/tasks/objectnav_mp3d.yaml")
config.defrost()
config.DATASET.SCENES_DIR = '/home/azav/habitat-api/data/scene_datasets/'
config.DATASET.DATA_PATH = '/home/azav/habitat-api/data/datasets/objectnav/mp3d/v1/{split}/{split}.json.gz'
config.DATASET.SPLIT = 'val'
config.SIMULATOR.AGENT_0.SENSORS = ['RGB_SENSOR', 'DEPTH_SENSOR', 'SEMANTIC_SENSOR']
config.SIMULATOR.DEPTH_SENSOR.NORMALIZE_DEPTH = False
config.freeze()

In [3]:
config


Config({'SEED': 100, 'ENVIRONMENT': Config({'MAX_EPISODE_STEPS': 500, 'MAX_EPISODE_SECONDS': 10000000, 'ITERATOR_OPTIONS': Config({'CYCLE': True, 'SHUFFLE': True, 'GROUP_BY_SCENE': True, 'NUM_EPISODE_SAMPLE': -1, 'MAX_SCENE_REPEAT_EPISODES': -1, 'MAX_SCENE_REPEAT_STEPS': 10000, 'STEP_REPETITION_RANGE': 0.2})}), 'TASK': Config({'TYPE': 'ObjectNav-v1', 'SUCCESS_DISTANCE': 0.1, 'SENSORS': ['OBJECTGOAL_SENSOR', 'COMPASS_SENSOR', 'GPS_SENSOR'], 'MEASUREMENTS': ['DISTANCE_TO_GOAL', 'SUCCESS', 'SPL'], 'GOAL_SENSOR_UUID': 'objectgoal', 'POSSIBLE_ACTIONS': ['STOP', 'MOVE_FORWARD', 'TURN_LEFT', 'TURN_RIGHT', 'LOOK_UP', 'LOOK_DOWN'], 'ACTIONS': Config({'STOP': Config({'TYPE': 'StopAction'}), 'MOVE_FORWARD': Config({'TYPE': 'MoveForwardAction'}), 'TURN_LEFT': Config({'TYPE': 'TurnLeftAction'}), 'TURN_RIGHT': Config({'TYPE': 'TurnRightAction'}), 'LOOK_UP': Config({'TYPE': 'LookUpAction'}), 'LOOK_DOWN': Config({'TYPE': 'LookDownAction'}), 'TELEPORT': Config({'TYPE': 'TeleportAction'}), 'ANSWER': Con

In [4]:
env = habitat.Env(config=config)



2020-05-17 22:30:11,452 Initializing dataset ObjectNav-v1
2020-05-17 22:30:14,355 initializing sim Sim-v0
I0517 22:30:28.789825 6173 simulator.py:142] Loaded navmesh /home/azav/habitat-api/data/scene_datasets/mp3d/TbHJrupSAjP/TbHJrupSAjP.navmesh
2020-05-17 22:30:28,810 Initializing task ObjectNav-v1


In [5]:

print(env.reset()['objectgoal'])
print(env.reset()['objectgoal'])
print(env.reset()['objectgoal'])
print(env.reset()['objectgoal'])

[6]
[12]
[12]
[8]


In [19]:
print(env._dataset.goals_by_category.keys())

dict_keys(['2azQ1b91cZZ.glb_cabinet', '2azQ1b91cZZ.glb_sofa', '2azQ1b91cZZ.glb_table', '2azQ1b91cZZ.glb_chair', '2azQ1b91cZZ.glb_cushion', '2azQ1b91cZZ.glb_plant', '2azQ1b91cZZ.glb_counter', '2azQ1b91cZZ.glb_sink', '2azQ1b91cZZ.glb_picture', '2azQ1b91cZZ.glb_fireplace', '2azQ1b91cZZ.glb_toilet', '2azQ1b91cZZ.glb_towel', '2azQ1b91cZZ.glb_seating', '2azQ1b91cZZ.glb_chest_of_drawers', '2azQ1b91cZZ.glb_bed', '2azQ1b91cZZ.glb_shower', '2azQ1b91cZZ.glb_bathtub', '2azQ1b91cZZ.glb_clothes', '2azQ1b91cZZ.glb_stool', '8194nk5LbLH.glb_gym_equipment', '8194nk5LbLH.glb_tv_monitor', '8194nk5LbLH.glb_picture', '8194nk5LbLH.glb_counter', '8194nk5LbLH.glb_chair', '8194nk5LbLH.glb_cabinet', '8194nk5LbLH.glb_table', '8194nk5LbLH.glb_stool', '8194nk5LbLH.glb_plant', '8194nk5LbLH.glb_towel', '8194nk5LbLH.glb_sofa', 'EU6Fwq7SyZv.glb_cabinet', 'EU6Fwq7SyZv.glb_shower', 'EU6Fwq7SyZv.glb_picture', 'EU6Fwq7SyZv.glb_toilet', 'EU6Fwq7SyZv.glb_towel', 'EU6Fwq7SyZv.glb_sink', 'EU6Fwq7SyZv.glb_chest_of_drawers', 'EU

In [14]:
print(env._dataset.category_to_task_category_id)
env._dataset.category_to_scene_annotation_category_id

{'chair': 0, 'table': 1, 'picture': 2, 'cabinet': 3, 'cushion': 4, 'sofa': 5, 'bed': 6, 'chest_of_drawers': 7, 'plant': 8, 'sink': 9, 'toilet': 10, 'stool': 11, 'towel': 12, 'tv_monitor': 13, 'shower': 14, 'bathtub': 15, 'counter': 16, 'fireplace': 17, 'gym_equipment': 18, 'seating': 19, 'clothes': 20}


{'chair': 3,
 'table': 5,
 'picture': 6,
 'cabinet': 7,
 'cushion': 8,
 'sofa': 10,
 'bed': 11,
 'chest_of_drawers': 13,
 'plant': 14,
 'sink': 15,
 'toilet': 18,
 'stool': 19,
 'towel': 20,
 'tv_monitor': 22,
 'shower': 23,
 'bathtub': 25,
 'counter': 26,
 'fireplace': 27,
 'gym_equipment': 33,
 'seating': 34,
 'clothes': 38}

In [21]:
scene_annotation_to_task_category_id = {}
for k, v in env._dataset.category_to_scene_annotation_category_id.items():
    scene_annotation_to_task_category_id[v] = env._dataset.category_to_task_category_id[k]
print(scene_annotation_to_task_category_id)  # same region(room?)+within FOV=GT, fix projection (check existing SLAM methods?)

{3: 0, 5: 1, 6: 2, 7: 3, 8: 4, 10: 5, 11: 6, 13: 7, 14: 8, 15: 9, 18: 10, 19: 11, 20: 12, 22: 13, 23: 14, 25: 15, 26: 16, 27: 17, 33: 18, 34: 19, 38: 20}


In [6]:
obs = env.reset()

In [11]:
names = []
for i in range(900):
    names.append(env.episodes[i].object_category)
print(len(set(names)))
print(names)

21
['counter', 'cabinet', 'cushion', 'chair', 'table', 'table', 'chair', 'picture', 'cabinet', 'seating', 'chair', 'table', 'bed', 'bed', 'sink', 'gym_equipment', 'sink', 'bed', 'cabinet', 'seating', 'cabinet', 'cushion', 'table', 'plant', 'chair', 'cabinet', 'table', 'cushion', 'bathtub', 'table', 'counter', 'clothes', 'table', 'picture', 'plant', 'toilet', 'towel', 'clothes', 'cabinet', 'table', 'picture', 'stool', 'cushion', 'picture', 'picture', 'table', 'cushion', 'chair', 'cabinet', 'sink', 'table', 'chair', 'fireplace', 'cushion', 'toilet', 'chair', 'table', 'table', 'cushion', 'clothes', 'chair', 'cushion', 'seating', 'table', 'picture', 'cushion', 'stool', 'table', 'seating', 'picture', 'chair', 'cushion', 'cabinet', 'cushion', 'chair', 'sink', 'cabinet', 'chair', 'chair', 'cushion', 'stool', 'table', 'fireplace', 'table', 'table', 'counter', 'cabinet', 'shower', 'chest_of_drawers', 'counter', 'picture', 'cushion', 'cabinet', 'table', 'chair', 'chair', 'chair', 'table', 'sofa'

In [ ]:

rtra = env._sim.sample_navigable_point()
ang = np.random.rand() * 2 * np.pi
rrot = quaternion.from_euler_angles(np.array([0, ang, 0]))

In [ ]:
ang = np.random.rand() * 2 * np.pi
rrot = quaternion.from_euler_angles(np.array([0, ang, 0]))

In [ ]:
HFOV = 79
CAMERA_HEIGHT = 0.88
def topdown_position(tra, height=2.5):
    return np.array([tra[0], tra[1] + height, tra[2] + CAMERA_HEIGHT])

def get_imgs(tra, rot, height=3.0):
    td_tra = topdown_position(tra, height)
    euler_angles = np.array([np.pi/2, np.pi/2, -np.pi/2 + ang])
    td_rot = quaternion.from_euler_angles(euler_angles)
    obs = env._sim.get_observations_at(position=tra, rotation=rot, keep_agent_at_new_pose=True)
    td_obs = env._sim.get_observations_at(position=td_tra, rotation=td_rot, keep_agent_at_new_pose=False)
    return obs['rgb'], obs['depth'], td_obs['semantic']

r, d, s = get_imgs(rtra, rrot, 5)
plt.imshow(r)
plt.show()
plt.imshow(d[:, :, 0])
plt.show()
print(np.max(d))

In [ ]:
fsm = Image.open('./mask80deg.png')
#mask = torch.tensor(np.array(Image.open('./mask80deg.png'))[:, :, 0] / 255, requires_grad=False)

In [ ]:
HFOV = 79
CAMERA_HEIGHT = 0.88

def topdown_position(tra, height=2.5):
    return np.array([tra[0], tra[1] + height, tra[2] + CAMERA_HEIGHT])
    
def color_semantic(semantic_obs):
    semantic_img = Image.new("P", (semantic_obs.shape[1], semantic_obs.shape[0]))
    semantic_img.putpalette(d3_40_colors_rgb.flatten())
    semantic_img.putdata((semantic_obs.flatten() % 40).astype(np.uint8))
    semantic_img = semantic_img.convert("RGBA")
    return semantic_img

def get_imgs(tra, rot, height=3.0):
    td_tra = topdown_position(tra, height)
    euler_angles = np.array([np.pi/2, np.pi/2, -np.pi/2 + rot[1]])
    td_rot = quaternion.from_euler_angles(euler_angles)
    rot = quaternion.from_euler_angles(rot)
    obs = env._sim.get_observations_at(position=tra, rotation=rot, keep_agent_at_new_pose=True)
    td_obs = env._sim.get_observations_at(position=td_tra, rotation=td_rot, keep_agent_at_new_pose=False)
    return obs['rgb'], obs['depth'], td_obs['semantic']
    
def mask_to_fov(img, ang, fov_shape_mask):
    #mask = torchrot(fov_shape_mask, ang * 180 / np.pi)
    mask = torch.tensor(np.array(fov_shape_mask, dtype=bool), requires_grad=False)[:, :, 0].float()
    
    return img * mask

def to_categories(object_ids, cat_id_map):
    for i in range(object_ids.shape[0]):
        for j in range(object_ids.shape[1]):
            object_ids[i][j] = cat_id_map[int(object_ids[i][j])] + 2

In [ ]:
#tds = torch.tensor(np.array(color_semantic(tds))[:, :, :-1], requires_grad=False).permute(2, 0, 1).unsqueeze(0).float() / 255


In [ ]:
model = WarpSeg(43)
optimizer = torch.optim.Adam(model.parameters(), weight_decay = 1e-5,  lr=0.001)
loss_func = nn.CrossEntropyLoss().to(device)

In [ ]:
d.shape

In [ ]:
bs = 64
n_scenes = 1000

start = time.time()
for scene_i in range(n_scenes):
    obs = env.reset()
    scene = env.sim.semantic_annotations()
    object_id_to_class_id = {int(obj.id.split("_")[-1]): obj.category.index() for obj in scene.objects}
    
    inputs = torch.zeros(bs, 4, 480, 640).to(device)
    targets = torch.zeros(bs, 64, 64).long()
    
    for i in range(bs):
        rtra = env._sim.sample_navigable_point()
        ang = np.random.rand() * 2 * np.pi
        rrot = np.array([0, ang, 0]))
        
        rgb, d, tds = get_imgs(rtra, rrot, 5)
        tds = torch.tensor(tds.astype(np.float32), requires_grad=False).unsqueeze(0).unsqueeze(0)
        tds = F.interpolate(tds[:, :, :, 80:-80], (64, 64), mode='nearest')
        to_categories(tds[0][0], object_id_to_class_id)
        masked = mask_to_fov(tds, ang, fsm)
        
        inputs[i, :3] = tensor(rgb / 255.).permute(2, 0, 1)
        inputs[i, 3] = tensor(d)[:, :, 0]
        targets[i] = masked[0][0]
        
    targets = targets.to(device)
    optimizer.zero_grad()
    preds = model(inputs)
    loss = loss_func(preds, targets)
    loss.backward()
    optimizer.step()
    
    if scene_i % 10 == 0:
        print('\r minibatch %d : trainloss %.4f time %.2f' % 
                      (scene_i + 1, float(loss.data), time.time() - start))